# Sans2d Data Reduction

## Introduction

This notebook gives an overview of how to use the `esssans` package with Sciline, on the example of the data reduction of a Sans2d experiment.
We begin with relevant imports:

In [ ]:
import scipp as sc
import plopp as pp
from ess import sans
from ess import isissans as isis
import ess.isissans.data  # noqa: F401
from ess.sans.types import *

## Create and configure the workflow

We begin by creating the Sans2d workflow object (this is a [sciline.Pipeline](https://scipp.github.io/sciline/generated/classes/sciline.Pipeline.html) which can be consulted for advanced usage).
The Sans2d workflow uses Mantid to load files.
This tutorial comes with files that do not require Mantid, so we use a slightly modified workflow that does not require Mantid.
The workflow is otherwise identical to the full Mantid-based workflow:

In [ ]:
workflow = isis.sans2d.Sans2dTutorialWorkflow()
# For real data use:
# workflow = isis.sans2d.Sans2dWorkflow()

We can insert steps for configuring the workflow.
In this case, we would like to use the transmission monitor from the regular background and sample runs since there was no separate transmission run.
We also want to compute the beam center using a simple center-of-mass estimation:

In [ ]:
workflow.insert(isis.io.transmission_from_background_run)
workflow.insert(isis.io.transmission_from_sample_run)

The workflow can be visualized as a graph.
For readability we show only sub-workflow for computing `IofQ[Sample]`.
The workflow can actually compute the full `BackgroundSubtractedIofQ`, which applies and equivalent workflow to the background run, before a subtraction step:

In [ ]:
# left-right layout works better for this graph
workflow.visualize(IofQ[SampleRun], graph_attr={"rankdir": "LR"})

Note the red boxes which indicate missing input parameters.
We can set these missing parameters, as well as parameters where we do not want to use the defaults:

In [ ]:
workflow[OutFilename] = 'reduced.nxs'

workflow[NeXusMonitorName[Incident]] = 'monitor2'
workflow[NeXusMonitorName[Transmission]] = 'monitor4'

workflow[isis.SampleOffset] = sc.vector([0.0, 0.0, 0.053], unit='m')
workflow[isis.MonitorOffset[Transmission]] = sc.vector([0.0, 0.0, -6.719], unit='m')

workflow[WavelengthBins] = sc.linspace(
    'wavelength', start=2.0, stop=16.0, num=141, unit='angstrom'
)

workflow[isis.sans2d.LowCountThreshold] = sc.scalar(100, unit='counts')

mask_interval = sc.array(dims=['wavelength'], values=[2.21, 2.59], unit='angstrom')
workflow[WavelengthMask] = sc.DataArray(
    sc.array(dims=['wavelength'], values=[True]),
    coords={'wavelength': mask_interval},
)

workflow[QBins] = sc.linspace(dim='Q', start=0.01, stop=0.6, num=141, unit='1/angstrom')
workflow[NonBackgroundWavelengthRange] = sc.array(
    dims=['wavelength'], values=[0.7, 17.1], unit='angstrom'
)
workflow[CorrectForGravity] = True
workflow[UncertaintyBroadcastMode] = UncertaintyBroadcastMode.upper_bound
workflow[ReturnEvents] = True

## Configuring data to load

We have not configured which files we want to load.
In this tutorial, we use helpers to fetch the tutorial data which return the filenames of the cached files.
In a real use case, you would set these parameters manually:

In [ ]:
workflow[DirectBeamFilename] = isis.data.sans2d_tutorial_direct_beam()
workflow[Filename[SampleRun]] = isis.data.sans2d_tutorial_sample_run()
workflow[Filename[BackgroundRun]] = isis.data.sans2d_tutorial_background_run()
workflow[Filename[EmptyBeamRun]] = isis.data.sans2d_tutorial_empty_beam_run()

The beam center is not set yet.
Unless we have a previously computed value, we can do so now:

In [ ]:
center = sans.beam_center_from_center_of_mass(workflow)
center

Remember to update the workflow with the new center:

In [ ]:
workflow[BeamCenter] = center

## Use the workflow

### Compute final result

We can now compute the background-subtracted $I(Q)$:

In [ ]:
result = workflow.compute(BackgroundSubtractedIofQ)
result.hist().plot(scale={'Q': 'log'}, norm='log')

As the result was computed in event-mode, we can also use a different $Q$-binning, without re-reducing the data:

In [ ]:
result.hist(Q=60).plot(scale={'Q': 'log'}, norm='log')

In the above we used an upper bound for the uncertainties of the normalization factors.
We can also compute the result with dropped normalization-factor uncertainties.
This is incorrect, but is useful for understanding whether the normalization factors significantly contribute to the uncertainty of the result:

In [ ]:
workflow[UncertaintyBroadcastMode] = UncertaintyBroadcastMode.drop
result_drop = workflow.compute(BackgroundSubtractedIofQ)
# Reset the UnsertaintyBroadcastMode to the old value
workflow[UncertaintyBroadcastMode] = UncertaintyBroadcastMode.upper_bound
sc.DataGroup(upper_bound=result, dropped=result_drop).hist().plot(norm='log')

### Save reduced data to file

`ess.sans` provides a function for saving the reduced data as an [NXcanSAS](https://manual.nexusformat.org/classes/applications/NXcanSAS.html) file.
It could be used directly with the `result` computed above, but we would have to provide the required metadata ourselves.
Instead, we use Sciline to get all required information directly from the pipeline: (See also the [File output](https://scipp.github.io/sciline/recipes/recipes.html#File-output) docs.)

In [ ]:
from ess.sans.io import save_background_subtracted_iofq

workflow.bind_and_call(save_background_subtracted_iofq)

### Compute intermediate results

For inspection and debugging purposes we can also compute intermediate results.
To avoid repeated computation (including costly loading of files) we can request multiple results at once, including the final result, if desired.
For example:

In [ ]:
monitors = (
    WavelengthMonitor[SampleRun, Incident],
    WavelengthMonitor[SampleRun, Transmission],
    WavelengthMonitor[BackgroundRun, Incident],
    WavelengthMonitor[BackgroundRun, Transmission],
)
parts = (
    WavelengthScaledQ[SampleRun, Numerator],
    WavelengthScaledQ[SampleRun, Denominator],
)
iofqs = (IofQ[SampleRun], IofQ[BackgroundRun], BackgroundSubtractedIofQ)
keys = (*monitors, MaskedData[SampleRun], *parts, *iofqs)

results = workflow.compute(keys)

display(sc.plot({str(key): results[key] for key in monitors}, norm='log'))

display(
    isis.plot_flat_detector_xy(
        results[MaskedData[SampleRun]]['spectrum', :61440].hist(), norm='log'
    )
)

wavelength = workflow.compute(WavelengthBins)
display(
    results[WavelengthScaledQ[SampleRun, Numerator]]
    .hist(wavelength=wavelength)
    .transpose()
    .plot(norm='log')
)
display(results[WavelengthScaledQ[SampleRun, Denominator]].plot(norm='log'))
parts = {str(key): results[key].sum('wavelength') for key in parts}
display(sc.plot(parts, norm='log'))

iofqs = {str(key): results[key] for key in iofqs}
iofqs = {key: val if val.bins is None else val.hist() for key, val in iofqs.items()}
display(sc.plot(iofqs, norm='log'))

## Wavelength bands

We can also compute $I(Q)$ inside a set of wavelength bands, instead of using the full wavelength range in one go.
This is useful for debugging purposes.

To achieve this, we need to supply the `WavelengthBands` parameter (as a two-dimensional variable),
representing the wavelength range for each band.

In [ ]:
workflow[WavelengthBands] = sc.linspace(
    'wavelength', start=2.0, stop=16.0, num=11, unit='angstrom'
)

Compute the result:

In [ ]:
result = workflow.compute(BackgroundSubtractedIofQ)
result

The result is two-dimensional and we over-plot all the bands onto the same axes:

In [ ]:
pp.plot(sc.collapse(result.hist(), keep='Q'), norm='log')